In [1]:
pip install git+https://github.com/fiaconis/Compy

  Cloning https://github.com/fiaconis/Compy to c:\users\valen\appdata\local\temp\pip-req-build-2m7vdwkt
  Resolved https://github.com/fiaconis/Compy to commit be0a8082dc10beab259d5b1d3d6e0a5c1185eb77
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 14.8/14.8 MB 1.0 MB/s eta 0:00:00
  Created wheel for compy: filename=compy-0.1.2-py3-none-any.whl size=7644 sha256=132a880a3d5cb83c33efde0ff8ec98a744795e587c9ed02a66b788349798f8a0
  Stored in directory: C:\Users\valen\AppData\Local\Temp\pip-ephem-wheel-cache-50vjqi4m\wheels\4f\68\d7\95290d6a40157e1d843624a2869776cd70d0eb8785ae3257fe
Successfully built compy
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/fiaconis/Compy 'C:\Users\valen\AppData\Local\Temp\pip-req-build-2m7vdwkt'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pingouin 0.5.2 requires scikit-learn<1.1.0, but you have scikit-learn 1.2.0 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.
emd 0.5.5 requires numpy<1.22, but you have numpy 1.24.2 which is incompatible.


In [2]:
import compy as cpy

'0.1.1'

In [10]:

from MFDFA import MFDFA
import pyedflib
import glob as gl
import matplotlib.image as mpimg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from numpy.fft import fft, fftfreq
from scipy import signal
import matplotlib.pyplot as plt
from mne.time_frequency.tfr import morlet
from mne.viz import plot_filter, plot_ideal_filter
import mne
import pandas as pd
import os 
   

In [6]:
def estetica(figura,x_name = '',y_name = '',title = '',w = 400,h = 325):
    
    figura.update_xaxes(showgrid=True,showline=True, linewidth=2, linecolor='black', mirror=True,title = x_name)#'cccccccccccccccc')
    figura.update_yaxes(showgrid=True,showline=True, linewidth=2, linecolor='black', mirror=True,title =y_name)


    figura.update_layout(
        margin=dict(l=20, r=20, t=40, b=20),
        plot_bgcolor="white",
        template="plotly_white", 
        title=title,
        autosize=False,
        width=w,
        height=h,
        font= dict(family="Arial",size=18,color="Black"),
        #yaxis = dict(range=[0,1]),
        #xaxis = dict(range=[0,1]),
        title_font = dict(size =20),
        legend=dict(orientation = 'v',
                    font = dict(size = 4),
                     yanchor="top",
                     y=0.95,
                     xanchor="right",
                     x=1,
                     bgcolor='rgba(255,255,255,0)',
                   )
    )
    
    
def filtroFIR(sig,fp,fl):
    
    import mne
    import numpy as np
    sfreq = 256
    f_p = fp
    f_l = fl
    transition_band = 0.25 * f_p
    f_s = f_p + transition_band


    h = mne.filter.create_filter(sig, sfreq, l_freq=f_l, h_freq=f_p,
                                 fir_design='firwin2', verbose=True)
    sig_filtrada = np.convolve(h, sig)[len(h) // 2:]
    
    return sig_filtrada


def derivar(sig):
    
    import numpy as np
    
    sigbufsdif = np.zeros(17,(len(sig[1])-1))
    for canal in range(17):    
        for i in range(len(sig[canal])-1):
            sigbufsdif[canal,i]=(sig[canal][i + 1] - sig[canal][i])
 
 
def leer_con_protocolo_10_20(dir,fp,fl):
    import pyedflib as pyedf
    
    f = pyedf.EdfReader(dir)
    signal_labels = f.getSignalLabels()
    sigbufs = np.zeros((17, f.getNSamples()[0]+3379))
    
    sigbufs[0, :] = filtroFIR((f.readSignal(0)-f.readSignal(1)),fp,fl) #Fp1-F3
    sigbufs[1, :] = filtroFIR((f.readSignal(1)-f.readSignal(2)),fp,fl)#F3-C3
    sigbufs[2, :] =filtroFIR((f.readSignal(2)-f.readSignal(3)),fp,fl) #C3-P3
    sigbufs[3, :] =filtroFIR((f.readSignal(3)-f.readSignal(4)),fp,fl) #P3-O1
    sigbufs[4, :] =filtroFIR((f.readSignal(0)-f.readSignal(5)),fp,fl) #Fp1-F7
    sigbufs[5, :] =filtroFIR((f.readSignal(5)-f.readSignal(6)),fp,fl) #F7-T3
    sigbufs[6, :] =filtroFIR((f.readSignal(6)-f.readSignal(7)),fp,fl) #T3-T5
    sigbufs[7, :] =filtroFIR((f.readSignal(7)-f.readSignal(4)),fp,fl) #T5-O1
    sigbufs[8, :] =filtroFIR((f.readSignal(19)),fp,fl) #Cz-Ref
    sigbufs[9, :] =filtroFIR((f.readSignal(9)-f.readSignal(10)),fp,fl) #Fp2-F4
    sigbufs[10, :] =filtroFIR((f.readSignal(10)-f.readSignal(11)),fp,fl) #F4-C4
    sigbufs[11, :] =filtroFIR((f.readSignal(11)-f.readSignal(12)),fp,fl) #C4-P4
    sigbufs[12, :] =filtroFIR((f.readSignal(12)-f.readSignal(13)),fp,fl) #P4-O2
    sigbufs[13, :] =filtroFIR((f.readSignal(9)-f.readSignal(14)),fp,fl) #Fp2-F8
    sigbufs[14, :] =filtroFIR((f.readSignal(14)-f.readSignal(15)),fp,fl) #F8-T4
    sigbufs[15, :] =filtroFIR((f.readSignal(15)-f.readSignal(16)),fp,fl)#T4-T6
    sigbufs[16, :] =filtroFIR((f.readSignal(16)-f.readSignal(4)),fp,fl) #T6-O1
    
    return sigbufs

            
def pasar_de_sig_a_df(sig,ch_names):
    import pandas as pd
    import numpy as np
    
    Data = pd.DataFrame({'t': np.arange(len(sig[0][:])),
                                ch_names[0]:sig[0][:],
                                ch_names[1]:sig[1][:],
                                ch_names[2]:sig[2][:],
                                ch_names[3]:sig[3][:],
                                ch_names[4]:sig[4][:],
                                ch_names[5]:sig[5][:],
                                ch_names[6]:sig[6][:],
                                ch_names[7]:sig[7][:],
                                ch_names[8]:sig[8][:],
                                ch_names[9]:sig[9][:],
                                ch_names[10]:sig[10][:],
                                ch_names[11]:sig[11][:],
                                ch_names[12]:sig[12][:],
                                ch_names[13]:sig[13][:],
                                ch_names[14]:sig[14][:],
                                ch_names[15]:sig[15][:],
                                ch_names[16]:sig[16][:]})
    return Data

def Hv_a_df(Hv):
    import pandas as pd
    import numpy as np
    
    acumulado= []
    df = pd.DataFrame()
    canal = 1
    for canal in range(len(Hv[1])):
        acumulado= []
        for i in range(len(Hv)):
            acumulado.append(Hv[i][canal])
        df[str(canal)]=acumulado
    acumulado= []        
    return df
    

    
def ventanteo_MFDFA(Data, ventana, salto, lag, q, order):
    import numpy as np
  
    Hv = []
    timeI1 = 0
    timeF1 = timeI1 + ventana
    inicio,final = int(timeI1*256),int(timeF1*256)
    
    while(final <= len(Data[0][:])):
        
        sig1 = []
        for canal in range(17):
            
            timeF1 = timeI1 + ventana
            inicio,final = int(timeI1*256),int(timeF1*256)
                
            sig1.append(Data[canal][inicio:final])
            
        H = np.zeros(17)
        for canal in range(17):    
            
            lag, dfa = MFDFA(np.asarray(sig1[canal]), lag = lag, q = q, order = order)
            Ho = np.polyfit(np.log(lag),np.log(dfa),1)[0]
            H[canal] = Ho
        Hv.append(H)
        timeI1 = timeI1 + salto
        
    return Hv


def guardar_df(Data, ubicacion_y_titulo):
    from pathlib import Path  
    filepath = Path(ubicacion_y_titulo)  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    Data.to_csv(filepath)  

    

In [284]:
ventana = 100
salto = 1
lag = np.unique(np.logspace(6, 11, 10, base = 2).astype(int))
q = 2
order = 1

fp = 30
fl = 0.25


Con= r"D:\Usuario\OneDrive\Escritorio\Uni\Pasantia\PacienteCon12.EDF"
Sin= r"D:\Usuario\OneDrive\Escritorio\Uni\Pasantia\PacienteSin8Guadalupe.EDF"

Con_Epilepcia= leer_con_protocolo_10_20(Con,fp,fl)
Sin_Epilepcia= leer_con_protocolo_10_20(Sin,fp,fl)


HvCP = ventanteo_MFDFA(Con_Epilepcia, ventana, salto, lag, q, order)
HvSP = ventanteo_MFDFA(Sin_Epilepcia, ventana, salto, lag, q, order)

Setting up band-pass filter from 0.25 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hamming window
- Lower passband edge: 0.25
- Lower transition bandwidth: 0.25 Hz (-6 dB cutoff frequency: 0.12 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 6759 samples (26.402 sec)

Setting up band-pass filter from 0.25 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hamming window
- Lower passband edge: 0.25
- Lower transition bandwidth: 0.25 Hz (-6 dB cutoff frequency: 0.12 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 6759 samples (26.402 sec)

Setting up band-pass filter from 0.25 - 30 Hz

FIR filter 

In [8]:
import plotly.graph_objects as go

canal = 3




df_HurstC,df_HurstS = Hv_a_df(HvCP),Hv_a_df(HvSP)
min_len = min(len(df_HurstC[str(canal)]),len(df_HurstS[str(canal)]))

plotC,plotS = df_HurstC[str(canal)][0:min_len],df_HurstS[str(canal)][0:min_len]

fig = go.Figure()
   
fig.add_trace(go.Scatter(x= np.arange(len(plotC)),y= plotC, mode='lines',name='Epilepticos',line_color='red',line_width=2.3))
fig.add_trace(go.Scatter(x= np.arange(len(plotS)),y= plotS, mode='lines',name='Neurotipicos',line_color='blue',line_width=2.3))

estetica(fig, h=450,w=700,x_name= 'Ventana',y_name='Hurst (2)')
fig.show()

hist = go.Figure()


hist.add_trace(go.Histogram(x=plotS))
hist.add_trace(go.Histogram(x=plotC))

estetica(hist, h=450,w=700)
hist.show()

NameError: name 'HvCP' is not defined

In [61]:
hist = go.Figure()

hist.add_trace(go.Histogram(x=plotS))
hist.add_trace(go.Histogram(x=plotC))

estetica(hist, h=450,w=700)
hist.show()

In [289]:
guardar_df(df_HurstC,r"D:\Usuario\OneDrive\Escritorio\Uni\NeuFiSur\DatosFiltrados\V100_P1_6-11\Con12_V100_P1.csv")
guardar_df(df_HurstS,r"D:\Usuario\OneDrive\Escritorio\Uni\NeuFiSur\DatosFiltrados\V100_P1_6-11\Sin8_V100_P1.csv")


#Revisar 6 Matheo

In [133]:
Con[7]


['D:\\Usuario\\OneDrive\\Escritorio\\Uni\\NeuFiSur\\DatosFiltrados\\Con10_V50_P1.csv',
 'D:\\Usuario\\OneDrive\\Escritorio\\Uni\\NeuFiSur\\DatosFiltrados\\Con11_V50_P1.csv',
 'D:\\Usuario\\OneDrive\\Escritorio\\Uni\\NeuFiSur\\DatosFiltrados\\Con12_V50_P1.csv',
 'D:\\Usuario\\OneDrive\\Escritorio\\Uni\\NeuFiSur\\DatosFiltrados\\Con13_V50_P1.csv',
 'D:\\Usuario\\OneDrive\\Escritorio\\Uni\\NeuFiSur\\DatosFiltrados\\Con14_V50_P1.csv',
 'D:\\Usuario\\OneDrive\\Escritorio\\Uni\\NeuFiSur\\DatosFiltrados\\Con15_V50_P1.csv',
 'D:\\Usuario\\OneDrive\\Escritorio\\Uni\\NeuFiSur\\DatosFiltrados\\Con16_V50_P1.csv',
 'D:\\Usuario\\OneDrive\\Escritorio\\Uni\\NeuFiSur\\DatosFiltrados\\Con5_V50_P1.csv',
 'D:\\Usuario\\OneDrive\\Escritorio\\Uni\\NeuFiSur\\DatosFiltrados\\Con6_V50_P1.csv',
 'D:\\Usuario\\OneDrive\\Escritorio\\Uni\\NeuFiSur\\DatosFiltrados\\Con7_V50_P1.csv',
 'D:\\Usuario\\OneDrive\\Escritorio\\Uni\\NeuFiSur\\DatosFiltrados\\Con8_V50_P1.csv',
 'D:\\Usuario\\OneDrive\\Escritorio\\Uni\\NeuFi

In [217]:
Sin[2]

's'

In [17]:
import glob as gl
canal = 8
fig = go.Figure()
hist = go.Figure()
Con = gl.glob(r"D:\Usuario\OneDrive\Escritorio\Uni\NeuFiSur\DatosFiltrados\V100_P1_6-11\Con*")
Sin = gl.glob(r"D:\Usuario\OneDrive\Escritorio\Uni\NeuFiSur\DatosFiltrados\V100_P1_6-11\Sin*")


paciente = 7


canal = 3




df_HurstCcsv = pd.read_csv(Con[3])
df_HurstScsv = pd.read_csv(Sin[2])

min_len = min(len(df_HurstCcsv[str(canal)]),len(df_HurstScsv[str(canal)]))

plotC,plotS = df_HurstCcsv[str(canal)][0:min_len],df_HurstScsv[str(canal)][0:min_len]

fig = go.Figure()
   
fig.add_trace(go.Scatter(x= np.arange(len(plotC)),y= plotC, mode='lines',name='Epilepticos',line_color='red',line_width=2.3))
fig.add_trace(go.Scatter(x= np.arange(len(plotS)),y= plotS, mode='lines',name='Neurotipicos',line_color='blue',line_width=2.3))

estetica(fig, h=450,w=700,x_name= 'Ventana',y_name='Hurst (2)')
fig.show()

hist = go.Figure()

hist.add_trace(go.Histogram(x=plotS))
hist.add_trace(go.Histogram(x=plotC))

estetica(hist, h=450,w=700)
hist.show()


In [22]:
plotS/plotS.sum()

0       0.000652
1       0.000651
2       0.000651
3       0.000646
4       0.000651
          ...   
1290    0.000787
1291    0.000782
1292    0.000784
1293    0.000787
1294    0.000785
Name: 3, Length: 1295, dtype: float64

In [66]:
hist, bin_edges = np.histogram(plotC, bins=20, range=[0.1,1.0])

In [67]:
bin_edges
bin = bin_edges[1:]-(bin_edges[1]-bin_edges[0])/2
bin

array([0.1225, 0.1675, 0.2125, 0.2575, 0.3025, 0.3475, 0.3925, 0.4375,
       0.4825, 0.5275, 0.5725, 0.6175, 0.6625, 0.7075, 0.7525, 0.7975,
       0.8425, 0.8875, 0.9325, 0.9775])

In [68]:
hist

array([  0,   0,   0,   0,   0,   0,   0,   0,  12,  40,  12,   5,  87,
       182, 202, 265, 325, 147,  18,   0], dtype=int64)

In [69]:
distri_df =pd.DataFrame({'bins':bin,
                         'cant':hist/hist.sum()})

In [51]:
distri_df

,bins,cant
0,0.145,0.000000
1,0.235,0.000000
2,0.325,0.000000
3,0.415,0.000000
4,0.505,0.040154
5,0.595,0.013127
6,0.685,0.207722
7,0.775,0.360618
8,0.865,0.364479
9,0.955,0.013900


In [70]:
cpy.complex_entropy(distri_df)

c:\Users\valen\mne-python\1.3.0_0\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning:

divide by zero encountered in log

c:\Users\valen\mne-python\1.3.0_0\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning:

divide by zero encountered in log



,Sx,Sy,Qx,Qy,Cx,Cy,Fx,Fy
0,0.647807,NaN,0.479063,NaN,0.31034,NaN,0.085063,NaN


In [65]:
cpy.complex_entropy(distri_df)

c:\Users\valen\mne-python\1.3.0_0\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning:

divide by zero encountered in log

c:\Users\valen\mne-python\1.3.0_0\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning:

divide by zero encountered in log



,Sx,Sy,Qx,Qy,Cx,Cy,Fx,Fy
0,0.479558,NaN,0.652875,NaN,0.313091,NaN,0.187518,NaN


In [94]:


for i in range(len(Con)):
    df_HurstC = pd.read_csv(Con[i])
    
    plotC,plotS = df_HurstC[str(canal)][0:min_len],df_HurstS[str(canal)][0:min_len]

    fig.add_trace(go.Scatter(x= np.arange(len(plotC)),y= plotC, mode='lines',name='Epilepticos',line_color='red',line_width=2.3))    
    hist.add_trace(go.Histogram(x=plotC, marker_color='red'))
    
for i in range(len(Sin)):
    df_HurstS = pd.read_csv(Sin[i])
    
    plotC,plotS = df_HurstC[str(canal)][0:min_len],df_HurstS[str(canal)][0:min_len]
    
    fig.add_trace(go.Scatter(x= np.arange(len(plotS)),y= plotS, mode='lines',name='Neurotipicos',line_color='blue',line_width=2.3))  
    hist.add_trace(go.Histogram(x=plotS, marker_color='blue'))
estetica(fig, h=600,w=800,x_name= 'Ventana',y_name='Hurst (2)')     
fig.show()






estetica(hist, h=800,w=1500)
hist.show()

In [ ]:
import numpy as np


ventana = 50
salto = 50
lag = np.unique(np.logspace(6, 10, 10, base = 2).astype(int))
q = 2
order = 1


Con = r"D:\Usuario\OneDrive\Escritorio\Uni\Pasantia\PacienteCon8.EDF"


Con_Epilepcia = leer_con_protocolo_10_20(Con,30)


Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed frequency-domain design (firwin2) method
- Hamming window
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 227 samples (0.887 sec)

Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed frequency-domain design (firwin2) method
- Hamming window
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 227 samples (0.887 sec)

Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed frequency-domain design (firwin2) method
- Hamming window
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6

1315

In [ ]:
sig1 = []
sig2 = []
fpL =30
ventana = 100

timeI1 = 764 
timeF1 = timeI1 + ventana
timeI2 = 180 
timeF2 = timeI2 + ventana

inicio = int(timeI1*256)
final = int(timeF1*256)
inicio2 = int(timeI2*256)
final2 = int(timeF2*256)
for i in range(0,17):
    
    sig1.append(filtroFIR(sigbufs[i][inicio:final],fpL))
    sig2.append(filtroFIR(sigbufs[i][inicio2:final2],fpL))
    
sig1 =np.array(sig1)
sig2 =np.array(sig2)


Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed frequency-domain design (firwin2) method
- Hamming window
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 227 samples (0.887 sec)

Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed frequency-domain design (firwin2) method
- Hamming window
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 227 samples (0.887 sec)

Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed frequency-domain design (firwin2) method
- Hamming window
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6

In [ ]:
import numpy as np

sfreq = 256  # Frecuencia de muestreo


ch_names = ['Fp1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 
            'Fp1-F7', 'F7-T3', 'T3-T5', 'T5-O1','Cz-Ref' ,'Fp2-F4', 
            'F4-C4', 'C4-P4', 'P4-O2', 'Fp2-F8', 'F8-T4', 'T4-T6', 'T6-O1']

info = mne.create_info(ch_names=ch_names, sfreq=sfreq)

raw = mne.io.RawArray(sigbufs[0:17][:], info)

scalings = {'mag': 2, 'grad': 2}

raw.plot(n_channels=4, scalings=scalings, title='Data from arrays',
         show=True, block=True)



Creating RawArray with float64 data, n_channels=17, n_times=353280
    Range : 0 ... 353279 =      0.000 ...  1379.996 secs
Ready.
Using qt as 2D backend.
Channels marked as bad:
none


In [ ]:



ch_names = ['Fp1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 
            'Fp1-F7', 'F7-T3', 'T3-T5', 'T5-O1','Cz-Ref' ,'Fp2-F4', 
            'F4-C4', 'C4-P4', 'P4-O2', 'Fp2-F8', 'F8-T4', 'T4-T6', 'T6-O1']

info = mne.create_info(ch_names=ch_names, sfreq=sfreq)

raw = mne.io.RawArray(sig1[0:17][:], info)

# Scaling of the figure.
# For actual EEG/MEG data different scaling factors should be used.
scalings = {'mag': 2, 'grad': 2}


raw.plot(n_channels=4, scalings=scalings, title='Data from arrays',
         show=True, block=True)

NameError: name 'sfreq' is not defined

In [ ]:
lag = np.unique(np.logspace(6, 10, 10, base = 2).astype(int))
start = -20
stop = 20
num = abs(start) + stop + 1
qo = np.linspace(start,stop,num)
q = []
for dato in qo:
    if dato != 0:
        q.append(dato)

order = 1


Hs = []

In [ ]:
canal = 0


H = np.zeros(17)
for canal in range(17):    
    
    DAT = sig2[canal] 
    lag, dfa = MFDFA(np.asarray(DAT), lag = lag, q = 2, order = order)
    Ho = np.polyfit(np.log(lag),np.log(dfa),1)[0]
    H[canal] = Ho


In [ ]:
import mne


sample_dir = r"D:\Usuario\OneDrive\Escritorio\Uni\Pasantia\PacienteCon8.EDF"


f = pyedflib.EdfReader(sample_dir )
n = f.signals_in_file

signal_labels = f.getSignalLabels()
sigbufs = np.zeros((17, f.getNSamples()[0]))
sigbufs[0, :] = (f.readSignal(0)-f.readSignal(1)) #Fp1-F3
sigbufs[1, :] = (f.readSignal(1)-f.readSignal(2)) #F3-C3
sigbufs[2, :] =(f.readSignal(2)-f.readSignal(3)) #C3-P3
sigbufs[3, :] =(f.readSignal(3)-f.readSignal(4)) #P3-O1
sigbufs[4, :] =(f.readSignal(0)-f.readSignal(5)) #Fp1-F7
sigbufs[5, :] =(f.readSignal(5)-f.readSignal(6)) #F7-T3
sigbufs[6, :] =(f.readSignal(6)-f.readSignal(7)) #T3-T5
sigbufs[7, :] =(f.readSignal(7)-f.readSignal(4)) #T5-O1
sigbufs[8, :] =(f.readSignal(19)) #Cz-Ref
sigbufs[9, :] =(f.readSignal(9)-f.readSignal(10)) #Fp2-F4
sigbufs[10, :] =(f.readSignal(10)-f.readSignal(11)) #F4-C4
sigbufs[11, :] =(f.readSignal(11)-f.readSignal(12)) #C4-P4
sigbufs[12, :] =(f.readSignal(12)-f.readSignal(13)) #P4-O2
sigbufs[13, :] =(f.readSignal(9)-f.readSignal(14)) #Fp2-F8
sigbufs[14, :] =(f.readSignal(14)-f.readSignal(15)) #F8-T4
sigbufs[15, :] =(f.readSignal(15)-f.readSignal(16)) #T4-T6
sigbufs[16, :] =(f.readSignal(16)-f.readSignal(4)) #T6-O1

ventana = 50

sig1 = []
HvC = []
fpL =30

for j in range(int(len(sigbufs[0])/(256*ventana))):
    sig1 = []
    for i in range(0,17):
        timeI1 = ventana*j
        timeF1 = timeI1 + ventana

        inicio = int(timeI1*256)
        final = int(timeF1*256)
            
        sig1.append(filtroFIR(sigbufs[i][inicio:final],fpL))
        H = np.zeros(17)
    
    for canal in range(17):    
        
        DAT = sig1[canal] 
        lag, dfa = MFDFA(np.asarray(DAT), lag = lag, q = 2, order = order)
        Ho = np.polyfit(np.log(lag),np.log(dfa),1)[0]
        H[canal] = Ho
    HvC.append(H)    
        


Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed frequency-domain design (firwin2) method
- Hamming window
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 227 samples (0.887 sec)

Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed frequency-domain design (firwin2) method
- Hamming window
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 227 samples (0.887 sec)

Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed frequency-domain design (firwin2) method
- Hamming window
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6

In [ ]:
sample_dir = r"D:\Usuario\OneDrive\Escritorio\Uni\Pasantia\PacienteSin3.EDF"


f = pyedflib.EdfReader(sample_dir )
n = f.signals_in_file

signal_labels = f.getSignalLabels()
sigbufs = np.zeros((17, f.getNSamples()[0]))
sigbufs[0, :] = (f.readSignal(0)-f.readSignal(1)) #Fp1-F3
sigbufs[1, :] = (f.readSignal(1)-f.readSignal(2)) #F3-C3
sigbufs[2, :] =(f.readSignal(2)-f.readSignal(3)) #C3-P3
sigbufs[3, :] =(f.readSignal(3)-f.readSignal(4)) #P3-O1
sigbufs[4, :] =(f.readSignal(0)-f.readSignal(5)) #Fp1-F7
sigbufs[5, :] =(f.readSignal(5)-f.readSignal(6)) #F7-T3
sigbufs[6, :] =(f.readSignal(6)-f.readSignal(7)) #T3-T5
sigbufs[7, :] =(f.readSignal(7)-f.readSignal(4)) #T5-O1
sigbufs[8, :] =(f.readSignal(19)) #Cz-Ref
sigbufs[9, :] =(f.readSignal(9)-f.readSignal(10)) #Fp2-F4
sigbufs[10, :] =(f.readSignal(10)-f.readSignal(11)) #F4-C4
sigbufs[11, :] =(f.readSignal(11)-f.readSignal(12)) #C4-P4
sigbufs[12, :] =(f.readSignal(12)-f.readSignal(13)) #P4-O2
sigbufs[13, :] =(f.readSignal(9)-f.readSignal(14)) #Fp2-F8
sigbufs[14, :] =(f.readSignal(14)-f.readSignal(15)) #F8-T4
sigbufs[15, :] =(f.readSignal(15)-f.readSignal(16)) #T4-T6
sigbufs[16, :] =(f.readSignal(16)-f.readSignal(4)) #T6-O1

ventana = 50

sig2 = []
Hv = []
fpL =30

for j in range(int(len(sigbufs[0])/(256*ventana))):
    sig2 = []
    for i in range(0,17):
        timeI1 = ventana*j
        timeF1 = timeI1 + ventana

        inicio = int(timeI1*256)
        final = int(timeF1*256)
        
            
        sig2.append(filtroFIR(sigbufs[i][inicio:final],fpL))
        H = np.zeros(17)
    
    for canal in range(17):    
        
        DAT = sig2[canal] 
        lag, dfa = MFDFA(np.asarray(DAT), lag = lag, q = 2, order = order)
        Ho = np.polyfit(np.log(lag),np.log(dfa),1)[0]
        H[canal] = Ho
    Hv.append(H)  



Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed frequency-domain design (firwin2) method
- Hamming window
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 227 samples (0.887 sec)

Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed frequency-domain design (firwin2) method
- Hamming window
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 227 samples (0.887 sec)

Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed frequency-domain design (firwin2) method
- Hamming window
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6

In [ ]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()
ploteoC = []
ploteoS = []
canal = 8




for i in range(len(HvC)):
    ploteoC.append(HvC[i][canal])
    ploteoS.append(Hv[i][canal])
    
fig.add_trace(go.Scatter(x= np.arange(len(ploteoC)),y= ploteoC, mode='lines',name='Epilepticos',line_color='red',line_width=2.3))
fig.add_trace(go.Scatter(x= np.arange(len(ploteoS)),y= ploteoS, mode='lines',name='Neurotipicos',line_color='blue',line_width=2.3))

estetica(fig, h=450,w=700,x_name= 'Ventana',y_name='Hurst (2)')
fig.show()


In [ ]:
fig1 = go.Figure()
ploteoC, ploteoS = [],[]
VariacionC,VariacionS = np.zeros(17),np.zeros(17)


for canal in range(17):
    ploteoC, ploteoS = [],[]

    for i in range(len(Hv)):

        ploteoC.append(HvC[i][canal])
        ploteoS.append(Hv[i][canal])
        
    VariacionC[canal] = max(ploteoC)-min(ploteoC)
    VariacionS[canal] = max(ploteoS)-min(ploteoS)
    
fig1.add_trace(go.Scatter(x= np.arange(len(VariacionC)),y= VariacionC, mode='lines',name='Epilepticos',line_color='red',line_width=2.3))
fig1.add_trace(go.Scatter(x= np.arange(len(VariacionS)),y= VariacionS, mode='lines',name='Neurotipicos',line_color='blue',line_width=2.3))    

estetica(fig1, h=450,w=700,x_name= 'Canales',y_name='Variacion')
fig1.show()
    

In [ ]:

C , S = 0, 0
for canal in range(17):
    
    if (VariacionC[canal] > VariacionS[canal]):
        C = C+1
    else:
        S = S+1

print("Epipleptico: " + str(C) )
print("No Epipleptico: " + str(S) )
             

Epipleptico: 12
No Epipleptico: 5


In [ ]:
import glob as gl

Sin_Epilepcia= gl.glob(r"D:\Usuario\OneDrive\Escritorio\Uni\Pasantia\PacienteSin*.EDF")
Con_Epilepcia= gl.glob(r"D:\Usuario\OneDrive\Escritorio\Uni\Pasantia\PacienteCon*.EDF")

Casos = [Sin_Epilepcia,Con_Epilepcia]